In [1]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/josue/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/josue/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/josue/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/josue/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/josue/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/josue/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package

[nltk_data]    |   Package timit is already up-to-date!
[nltk_data]    | Downloading package toolbox to
[nltk_data]    |     /Users/josue/nltk_data...
[nltk_data]    |   Package toolbox is already up-to-date!
[nltk_data]    | Downloading package treebank to
[nltk_data]    |     /Users/josue/nltk_data...
[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     /Users/josue/nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to /Users/josue/nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     /Users/josue/nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     /Users/josue/nltk_data...
[nltk_data]    |   Package unicode_samples is already up-to-date!
[nltk_data]    | Downlo

True

## Reading File and creating DataFrame

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import pickle

In [3]:
import pandas as pd
wiki_files = pd.read_csv('wiki_sample.csv')
wiki_dataframe = pd.DataFrame(wiki_files)
wiki_dataframe

,content,title,id
0,Morocco–Saudi Arabia relations\r\n\r\nMoroccan...,Morocco–Saudi Arabia relations,1
1,Anthony United States Post Office\r\n\r\nThe A...,Anthony United States Post Office,2
2,Dumraon (Vidhan Sabha constituency)\r\n\r\nDum...,Dumraon Vidhan Sabha constituency,3
3,Chaker Khazaal\r\n\r\nChaker Khazaal (born 28 ...,Chaker Khazaal,4
4,Vicente Pascual Pastor\r\n\r\nVicente Pascual ...,Vicente Pascual Pastor,5
...,...,...,...
1286259,Toppr\r\n\r\nToppr is a product of Haygot Educ...,Toppr,1286260
1286260,Outline of education\r\n\r\nThe following outl...,Outline of education,1286261
1286261,Raell Padamsee\r\n\r\nRaell Padamsee is an Ind...,Raell Padamsee,1286262
1286262,Megan Callaway\r\n\r\nMegan Callaway is a foun...,Megan Callaway,1286263


In [9]:
sum = 0
for entry in wiki_dataframe['tokenized_content']:
    sum += len(entry)
    

KeyError: 'tokenized_content'

In [ ]:
sum
len(vocab)

In [ ]:
total_words
len(inv_idx)

## Preprocessing and Creating Vocab

In [4]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import pycountry
lemmatizer = WordNetLemmatizer()
from names_dataset import NameDatasetV1 # v1
names = NameDatasetV1()

stop_words = set(stopwords.words('english'))
# for domain_stop_word in domain_stop_words:
#     stop_words.add(domain_stop_word)

In [5]:
import pandas as pd
companies_file = pd.read_csv('companies_sorted.csv')
companies_dataframe = pd.DataFrame(companies_file)
companies_dataframe

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768
3,2309813,us army,goarmy.com,1800.0,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958
4,1558607,ey,ey.com,1989.0,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960
...,...,...,...,...,...,...,...,...,...,...,...
7173421,1494427,certiport vouchers,certiportvouchers.com,2011.0,information technology and services,1 - 10,NaN,NaN,linkedin.com/company/certiport-vouchers,0,1
7173422,1494429,black tiger fight club,blacktigerclub.com,2006.0,"health, wellness and fitness",1 - 10,"peking, beijing, china",china,linkedin.com/company/black-tiger-club-hero,0,6
7173423,4768462,catholic bishop of chicago,NaN,NaN,religious institutions,1 - 10,"inverness, illinois, united states",united states,linkedin.com/company/catholic-bishop-of-chicago,0,1
7173424,1494436,medexo robotics ltd,NaN,NaN,research,1 - 10,"london, london, united kingdom",united kingdom,linkedin.com/company/medexo-robotics-ltd,0,2


In [6]:
companies = set(companies_dataframe['name'])

In [7]:
stop_words = set(stopwords.words('english'))

In [8]:
words = set(nltk.corpus.words.words())
lowerCasedWords = map(lambda word: word.lower(), nltk.corpus.words.words())
lowerCasedWords = set(list(lowerCasedWords))
for country in list(pycountry.countries):
    lowerCasedWords.add(country.name.lower())
for company in companies:
    lowerCasedWords.add(str(company).lower())

In [9]:
# lemmatization, lowercase, remove non alpha, remove non-english, remove numbers and stopword removal
rejected_content = []
def preprocess_vocab(row, lenTitle):
    filtered_content = []
    for token in nltk.word_tokenize(row['content'][lenTitle:]):
        token = lemmatizer.lemmatize(token).lower()
        if names.search_first_name(token) or names.search_last_name(token) or ((token in lowerCasedWords) and (token not in stop_words) and (token.isalpha())):
            filtered_content.append(token) 
        else:
            rejected_content.append(token)
        
    return filtered_content

In [10]:
# no preprocessing
rejected_content = []
def preprocess_vocab(row, lenTitle):
    filtered_content = []
    for token in nltk.word_tokenize(row['content'][lenTitle:]):
        filtered_content.append(token) 
    return filtered_content

In [11]:
# just lemmatizing
rejected_content = []
def preprocess_vocab_lemmatize(row, lenTitle):
    filtered_content = []
    for token in nltk.word_tokenize(row['content'][lenTitle:]):
        token = lemmatizer.lemmatize(token)
        filtered_content.append(token) 
    return filtered_content

In [12]:
# just lowercasing
rejected_content = []
def preprocess_vocab_lowercase(row, lenTitle):
    filtered_content = []
    for token in nltk.word_tokenize(row['content'][lenTitle:]):
        token = token.lower()
        filtered_content.append(token) 
    return filtered_content

In [13]:
# just alpha filtering
rejected_content = []
def preprocess_vocab_alphafilter(row, lenTitle):
    filtered_content = []
    for token in nltk.word_tokenize(row['content'][lenTitle:]):
        if token.isalpha():
            filtered_content.append(token) 
    return filtered_content

In [14]:
# just stopword removal
rejected_content = []
def preprocess_vocab_stopwordremoval(row, lenTitle):
    filtered_content = []
    for token in nltk.word_tokenize(row['content'][lenTitle:]):
        if token not in stopwords:
            filtered_content.append(token) 
    return filtered_content

In [15]:
#just checking for first/last/country/company name
rejected_content = []
def preprocess_vocab_corpus(row, lenTitle):
    filtered_content = []
    for token in nltk.word_tokenize(row['content'][lenTitle:]):
        if names.search_first_name(token) or names.search_last_name(token) or (token in lowerCasedWords):
            filtered_content.append(token) 
    return filtered_content

In [19]:
wiki_dataframe['tokenized_content'] = wiki_dataframe.progress_apply(lambda row: preprocess_vocab(row, len(row['title'])), axis=1)

100%|████████████████████████████████| 1286264/1286264 [31:21<00:00, 683.50it/s]


In [20]:
from collections import defaultdict
from collections import Counter

inv_idx = defaultdict(list)
vocab = set()
heaps_law_dataset = list()

most_freq = []
id = 1
total_words = 0
for document in tqdm(wiki_dataframe['tokenized_content']):
    counter = Counter(document)
    most_occur = counter.most_common(1)
    most_freq.append(most_occur)
    heaps_law_dataset.append((total_words, len(vocab)))
    for word in document:
        inv_idx[word].append(id)
        total_words += 1
        vocab.add(word)
    id +=1
len(vocab)

100%|███████████████████████████████| 1286264/1286264 [03:36<00:00, 5930.19it/s]


2936354

In [16]:
wiki_dataframe['tokenized_content_lemmatize'] = wiki_dataframe.progress_apply(lambda row: preprocess_vocab_lemmatize(row, len(row['title'])), axis=1)

100%|██████████| 1286264/1286264 [41:04<00:00, 521.93it/s] 


In [17]:
from collections import defaultdict
from collections import Counter

inv_idx = defaultdict(list)
vocab = set()
heaps_law_dataset = list()

most_freq = []
id = 1
total_words = 0
for document in tqdm(wiki_dataframe['tokenized_content_lemmatize']):
    counter = Counter(document)
    most_occur = counter.most_common(1)
    most_freq.append(most_occur)
    heaps_law_dataset.append((total_words, len(vocab)))
    for word in document:
        inv_idx[word].append(id)
        total_words += 1
        vocab.add(word)
    id +=1
len(vocab)

100%|██████████| 1286264/1286264 [04:50<00:00, 4425.32it/s] 


2912918

In [18]:
wiki_dataframe['tokenized_content_lowercase'] = wiki_dataframe.progress_apply(lambda row: preprocess_vocab_lowercase(row, len(row['title'])), axis=1)

100%|██████████| 1286264/1286264 [30:53<00:00, 693.82it/s]  


In [19]:
from collections import defaultdict
from collections import Counter

inv_idx = defaultdict(list)
vocab = set()
heaps_law_dataset = list()

most_freq = []
id = 1
total_words = 0
for document in tqdm(wiki_dataframe['tokenized_content_lowercase']):
    counter = Counter(document)
    most_occur = counter.most_common(1)
    most_freq.append(most_occur)
    heaps_law_dataset.append((total_words, len(vocab)))
    for word in document:
        inv_idx[word].append(id)
        total_words += 1
        vocab.add(word)
    id +=1
len(vocab)

100%|██████████| 1286264/1286264 [18:38<00:00, 1150.10it/s] 


2666377

In [20]:
wiki_dataframe['tokenized_content_alphafilter'] = wiki_dataframe.progress_apply(lambda row: preprocess_vocab_alphafilter(row, len(row['title'])), axis=1)

 82%|████████▏ | 1051757/1286264 [24:07<05:22, 726.49it/s] 


KeyboardInterrupt: 

In [ ]:
from collections import defaultdict
from collections import Counter

inv_idx = defaultdict(list)
vocab = set()
heaps_law_dataset = list()

most_freq = []
id = 1
total_words = 0
for document in tqdm(wiki_dataframe['tokenized_content_alphafilter']):
    counter = Counter(document)
    most_occur = counter.most_common(1)
    most_freq.append(most_occur)
    heaps_law_dataset.append((total_words, len(vocab)))
    for word in document:
        inv_idx[word].append(id)
        total_words += 1
        vocab.add(word)
    id +=1
len(vocab)

In [ ]:
wiki_dataframe['tokenized_content_stopwordremoval'] = wiki_dataframe.progress_apply(lambda row: preprocess_vocab_stopwordremoval(row, len(row['title'])), axis=1)

In [ ]:
from collections import defaultdict
from collections import Counter

inv_idx = defaultdict(list)
vocab = set()
heaps_law_dataset = list()

most_freq = []
id = 1
total_words = 0
for document in tqdm(wiki_dataframe['tokenized_content_stopwordremoval']):
    counter = Counter(document)
    most_occur = counter.most_common(1)
    most_freq.append(most_occur)
    heaps_law_dataset.append((total_words, len(vocab)))
    for word in document:
        inv_idx[word].append(id)
        total_words += 1
        vocab.add(word)
    id +=1
len(vocab)

In [ ]:
wiki_dataframe['tokenized_content_corpus'] = wiki_dataframe.progress_apply(lambda row: preprocess_vocab_corpus(row, len(row['title'])), axis=1)

In [ ]:
from collections import defaultdict
from collections import Counter

inv_idx = defaultdict(list)
vocab = set()
heaps_law_dataset = list()

most_freq = []
id = 1
total_words = 0
for document in tqdm(wiki_dataframe['tokenized_content_corpus']):
    counter = Counter(document)
    most_occur = counter.most_common(1)
    most_freq.append(most_occur)
    heaps_law_dataset.append((total_words, len(vocab)))
    for word in document:
        inv_idx[word].append(id)
        total_words += 1
        vocab.add(word)
    id +=1
len(vocab)

In [ ]:
wiki_dataframe.to_pickle('./wiki_dataframe_data.pkl')

In [5]:
import pickle5 as pickle
with open("./data/wiki_dataframe.pkl", "rb") as pickle_file:
    wiki_dataframe = pickle.load(pickle_file)
# wiki_dataframe = pd.read_pickle('./wiki_dataframe_augmented_nltk_corpus_to_remove_non-english.pkl')

FileNotFoundError: [Errno 2] No such file or directory: './data/wiki_dataframe.pkl'

In [ ]:
wiki_dataframe

## Creating Part of Inverted Index

In [ ]:
from collections import defaultdict
from collections import Counter

inv_idx = defaultdict(list)
vocab = set()
heaps_law_dataset = list()

most_freq = []
id = 1
total_words = 0
for document in tqdm(wiki_dataframe['tokenized_content']):
    counter = Counter(document)
    most_occur = counter.most_common(1)
    most_freq.append(most_occur)
    heaps_law_dataset.append((total_words, len(vocab)))
    for word in document:
        inv_idx[word].append(id)
        total_words += 1
        vocab.add(word)
    id +=1
len(vocab)

In [ ]:
wiki_dataframe['most_frequent_term'] = most_freq

In [ ]:
wiki_dataframe

In [ ]:
from collections import OrderedDict
inv_idx_ordered = OrderedDict(sorted(inv_idx.items(), key=lambda item: len(item[1]), reverse=True))

In [ ]:
words = list(inv_idx_ordered.keys())
domain_stop_words = words[0:10000]

## Heaps Law

In [ ]:
heaps_law_dataset[len(heaps_law_dataset)-1]

In [ ]:
import matplotlib.pyplot as plt
import math

def heaps_law(list_to_graph):
        x = list()
        y = list()
        
        for item in list_to_graph:
            x.append(item[0])
            y.append(item[1])

        plt.plot(x, y)
        plt.xlim(1, x[-1])
        plt.ylim(1, y[-1])
        plt.savefig("heaps_law_words_from_nltk_english_corpus.png")

In [ ]:
heaps_law(heaps_law_dataset)

## Zipfs Law

In [ ]:
import matplotlib.pyplot as plt
import math

def zipfs_law(list_to_graph):
        x = list()
        y = list()
        
        for i, word in enumerate(list_to_graph):
            x.append(i+1)
            y.append(len(inv_idx_ordered[word]))
            

        plt.loglog(x, y)
        plt.savefig("zipfs_law.png")

In [ ]:
zipfs_law(inv_idx_ordered)

## Finishing Inverted Index

In [ ]:
for value in tqdm(inv_idx_ordered.items()):
    inv_idx[value[0]] = (Counter(value[1]))

In [ ]:
inv_idx

In [ ]:
file_to_write = open("inv_idx_augmented_nltk_corpus_to_remove_non-english.pkl", "wb")
pickle.dump(inv_idx, file_to_write)

In [5]:
inv_idx = pickle.load(open("inv_idx_augmented_nltk_corpus_to_remove_non-english.pkl", "rb"))

## Suggesting Queries

In [11]:
from os import listdir
from os.path import isfile, join
aol_query_log = pd.read_csv('project_1_AOL_query_log/Clean-Data-01.txt', sep="\t")
for file in listdir('project_1_AOL_query_log')[1:]:
    aol_query_log = aol_query_log.append(pd.read_csv('project_1_AOL_query_log/' + str(file), sep="\t"), ignore_index=False)

In [12]:
print(set(aol_query_log[aol_query_log['Query'] == 'christmas tree']['AnonID']))
print(set(aol_query_log[aol_query_log['Query'] == 'christmas tree shop']['AnonID']))
print(len(set(aol_query_log[aol_query_log['Query'] == 'christmas tree shop']['AnonID']) & set(aol_query_log[aol_query_log['Query'] == 'christmas tree']['AnonID'])))

{10710209, 4618828, 128728, 8584343, 8232760, 6356571, 13539708, 23577405}
{4013446, 1735558, 9887625, 10497418, 3030796, 1473164, 743055, 3322000, 8769041, 1139474, 8543763, 3874196, 4869267, 6326678, 2337176, 5175448, 14730651, 11888540, 11390626, 13650599, 8741929, 3234354, 9099830, 4262332, 3404229, 686022, 502471, 22863813, 3474638, 154448, 2272473, 1856732, 17403487, 5883615, 6637664, 588260, 2505191, 299377, 3851124, 795126, 14473982}
0


In [ ]:
set(aol_query_log[aol_query_log['Query'] == 'vietnam']['AnonID'])


In [ ]:
aol_query_log

In [ ]:
type(aol_query_log['AnonID'][0])

In [10]:
from collections import defaultdict
import itertools

aol_queries = aol_query_log['Query'].values
def identify_candidate_queries(query):
    print("Identifying Candidate Queries...")
    candidate_queries = defaultdict(int)
    potential = list()
    sessions_with_original_query = set(aol_query_log[aol_query_log['Query'] == query]['AnonID'])
    for anon_id in sessions_with_original_query:
        potential_queries = aol_query_log[aol_query_log['AnonID'] == anon_id]['Query']
        if len(potential_queries) > 1:
            potential.append(potential_queries)
    for aol_query in itertools.chain(*potential):
        if len(str(aol_query).split()) > len(query.split()):
            if aol_query.startswith(query):
                candidate_queries[aol_query] += 1
    print("Done!")
    return candidate_queries, len(sessions_with_original_query)
candidate_queries, num_sessions_containing_q = identify_candidate_queries("men")

Identifying Candidate Queries...
Done!


In [15]:
x = set(aol_query_log[aol_query_log['Query'] == 'christmas tree']['AnonID'])
y = set(aol_query_log[aol_query_log['Query'] == 'christmas tree shop']['AnonID'])
x.intersection(y)


set()

#### Ranking Candidate Suggestions
##### 𝑆𝑐𝑜𝑟𝑒(𝐶𝑄, 𝑞′) = \# 𝑜𝑓 𝑠𝑒𝑠𝑠𝑖𝑜𝑛𝑠 𝑖𝑛 𝑤ℎ𝑖𝑐ℎ 𝑞 𝑖𝑠 𝑚𝑜𝑑𝑖𝑓𝑖𝑒𝑑 𝑡𝑜 𝐶𝑄 ÷ \# 𝑜𝑓 𝑠𝑒𝑠𝑠𝑖𝑜𝑛𝑠 𝑖𝑛 𝑤ℎ𝑖𝑐ℎ 𝑞 𝑎𝑝𝑝𝑒𝑎𝑟𝑠

In [19]:
def rank_candidate_queries(original_query, candidates, num_sessions_containing_q):
    print("Ranking Candidate Queries")
    candidate_scores = []

    for candidate in tqdm(candidates): 
        if num_sessions_containing_q != 0:
            candidate_scores.append((candidate, candidates[candidate] / num_sessions_containing_q))
   
    if len(candidate_scores) < 1:
        candidate_scores.append((original_query, 0))
    print("Done!")
    return sorted(candidate_scores, key=lambda candidate_score: candidate_score[1], reverse=True)
rank_candidate_queries("men", candidate_queries, num_sessions_containing_q)

NameError: name 'candidate_queries' is not defined

In [20]:
def find_rank_candidate_queries(query):
    candidate_list, num_sessions_containing_q = identify_candidate_queries(query)
    return rank_candidate_queries(query, candidate_list, num_sessions_containing_q)

In [21]:
find_rank_candidate_queries("frozen")

NameError: name 'identify_candidate_queries' is not defined

## Relevance Ranking

### Identifying Candidate Resources

In [54]:
from nltk.corpus import wordnet
from itertools import combinations
def identify_candidate_resources(query):
    print("Identifying Candidate Resources...")
    results = set()
    split_query = query.split()
    synonyms = list()
    for term in split_query:
        synonyms.extend(wordnet.synsets(term))
    for word in synonyms:
        split_query.append(word.lemmas()[0].name())
    split_query = list(set(split_query))
    print(split_query)
    n = len(split_query)
    candidate_list = list()
    for term in split_query:
#         print(".", end="")
        if len(inv_idx[term]) > 0:
            candidate_list.append(set(inv_idx[term].keys()))
#     print(candidate_list[0])
    if len(candidate_list) > 0:
        results = set.intersection(*candidate_list)
    if len(results) <= 50:
        for combination in combinations(split_query, n - 1):
#             print(".", end="")
            candidate_list = list()
            for term in combination:
                if len(inv_idx[term]) > 0:
                    candidate_list.append(set(inv_idx[term].keys()))
            if len(candidate_list) > 0:
                print(candidate_list)
                results = set.intersection(*candidate_list)
            if len(results) > 50:
                break
            else:
                n -= 1
    return results

In [55]:
identify_candidate_resources('sex')

Identifying Candidate Resources...
['arouse', 'sexual_activity', 'sex']
[{501249, 108035, 107532, 781328, 379929, 770590, 819745, 399404, 136241, 154673, 732210, 642100, 286776, 350266, 993343, 1060939, 74829, 548941, 1021520, 321116, 1030239, 1044577, 1029218, 336997, 217715, 1017464, 362619, 811653, 721547, 285836, 673943, 567963, 432285, 825501, 1086111, 452773, 513190, 469679, 222897, 1150641, 280251, 249538, 554185, 500939, 1182412, 1226955, 162513, 828640, 469223, 657643, 497903, 887538, 700661, 319224, 833277, 568574, 1206525, 444679, 65815, 127258, 580896, 1283364, 984363, 453936, 1016112, 81721, 611645, 24902, 538968, 344931, 1080166, 408423, 735602, 873341, 335745, 63378, 506264, 283549, 394656, 164261, 715693, 316337, 683956, 361910, 189368, 645052, 1029564, 1124286, 232899, 479687, 278473, 1110478, 480727, 669145, 16860, 75250}]


{16860,
 24902,
 63378,
 65815,
 74829,
 75250,
 81721,
 107532,
 108035,
 127258,
 136241,
 154673,
 162513,
 164261,
 189368,
 217715,
 222897,
 232899,
 249538,
 278473,
 280251,
 283549,
 285836,
 286776,
 316337,
 319224,
 321116,
 335745,
 336997,
 344931,
 350266,
 361910,
 362619,
 379929,
 394656,
 399404,
 408423,
 432285,
 444679,
 452773,
 453936,
 469223,
 469679,
 479687,
 480727,
 497903,
 500939,
 501249,
 506264,
 513190,
 538968,
 548941,
 554185,
 567963,
 568574,
 580896,
 611645,
 642100,
 645052,
 657643,
 669145,
 673943,
 683956,
 700661,
 715693,
 721547,
 732210,
 735602,
 770590,
 781328,
 811653,
 819745,
 825501,
 828640,
 833277,
 873341,
 887538,
 984363,
 993343,
 1016112,
 1017464,
 1021520,
 1029218,
 1029564,
 1030239,
 1044577,
 1060939,
 1080166,
 1086111,
 1110478,
 1124286,
 1150641,
 1182412,
 1206525,
 1226955,
 1283364}

### TF-IDF
#### 𝑇𝐹(𝑤, 𝑑) = 𝑓𝑟𝑒𝑞(𝑤, 𝑑) ÷ (𝑚𝑎𝑥_𝑑)
#### 𝐼𝐷𝐹(𝑤) = 𝑙𝑜𝑔__2 (𝑁 ÷ 𝑛_𝑤)

In [22]:
len(wiki_dataframe)

1286264

In [23]:
import math
def tf_idf(split_query, document_id):
    score = 0

    if document_id == 0:
        print("error in tf-idf function")
        return

    for term in split_query:
        if term not in inv_idx:
            continue
        score += (inv_idx[term][document_id] / wiki_dataframe['most_frequent_term'][document_id - 1][0][1]) * math.log((len(wiki_dataframe) / len(inv_idx[term])), 2)
    return score

### Ranking candidate resources

In [24]:
def rank_candidate_resources(query, candidate_resources):
  ranked_candidates = {}
  for document_id in candidate_resources:
    ranked_candidates[document_id] = tf_idf(query.split(), document_id)
  return sorted(ranked_candidates.items(), key=lambda item: item[1], reverse=True)

In [25]:
def find_and_rank_candidate_resources(query):
    candidates = identify_candidate_resources(query)
    return rank_candidate_resources(query, candidates)
relevant_resources = find_and_rank_candidate_resources("sex")[0]
relevant_resources

Identifying Candidate Resources...
[Synset('sexual_activity.n.01'), Synset('sex.n.02'), Synset('sex.n.03'), Synset('sex.n.04'), Synset('arouse.v.07'), Synset('sex.v.02')]
['arouse', 'sex', 'sexual_activity']
..........

TypeError: descriptor 'intersection' requires a 'set' object but received a 'Counter'

### Snippet Generation

In [15]:
def get_snippet(query, document_id):
    row = wiki_dataframe[wiki_dataframe['id'] == document_id + 1]
    snippet = (  # tuple in the form (title, sentences)
        row["title"].to_string(index=False),
        generate_sentence_snippets(query, document_id, int(row["title"].str.len())))
    return snippet

In [21]:
wiki_dataframe["title"][0]

'Morocco–Saudi Arabia relations'

In [21]:
import re
def generate_sentence_snippets(query, document_id, len_title):
    pattern = '(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s'
    top_two = []  # format is (sentence, cosine_similarity_score)
    vectorized_query = vectorize(query, document_id)
    for sentence in re.split(pattern, wiki_dataframe["content"][document_id][len_title:]):
        sentence = sentence.replace("\r", "").replace("\n", "")  # 4 is for removal of \r\n\r\n for each sentence.
        lsentence = sentence.lower()

        sentence_score = cosine_similarity(vectorized_query, vectorize(lsentence, document_id))
        if len(top_two) < 2:
            top_two.append((sentence, sentence_score))
            top_two.sort(key=lambda item: item[1], reverse=True)
            continue

        for index, entry in enumerate(top_two):  # this loop should only run twice
            if sentence_score > entry[1]:
                top_two[index] = (sentence, sentence_score)
                top_two.sort(key=lambda item: item[1], reverse=True)
                break
    if len(top_two) < 2:
        result = top_two[0][0]
    else:
        result = (top_two[0][0], top_two[1][0])
    return result

In [18]:
def vectorize(phrase, document_id):
    arr = []
    for word in phrase.split():
        arr.append(tf_idf([word], document_id))
    return arr

In [19]:
from scipy import spatial
def cosine_similarity(query, sentence, doc_id):
    colab_sent = query | sentence
    print(colab_sent)
    vectorized_query = []
    vectorized_sentence = []
    for word in colab_sent:
        if word in query.split():
            vectorized_query.append(tf_idf([word]), doc_id)
        else:
            vectorized_query.append(0)
        if word in sentence:
            vectorized_sentence.append(tf_idf([word]), doc_id)
        else:
            vectorized_sentence.append(0)
    print(vectorized_query)
    print(vectorized_sentence)
    return 1 - spatial.distance.cosine(list(vectorized_sentence), list(vectorized_query))

### Putting Everything together

In [3]:
import pickle5 as pickle

def load_files():
    print("loading files...")
    with open("./data/wiki_dataframe_augmented_nltk_corpus_to_remove_non-english.pkl", "rb") as wiki_df:
        wiki_dataframe = pickle.load(wiki_df)
    inv_idx = pickle.load(open("./data/inv_idx_augmented_nltk_corpus_to_remove_non-english.pkl", "rb"))
    aol_query_log = pickle.load(open("./data/aol_query_log_data.pkl", "rb"))
    print("Done!")
    return [wiki_dataframe, inv_idx, aol_query_log]


wiki_dataframe, inv_idx, aol_query_log = load_files()


loading files...
Done!


In [4]:
len(inv_idx)

284750

In [16]:
def search(query):
    # print("Generating results...")
    # ranked_candidate_resources = find_and_rank_candidate_resources(query)[0:10]
    results = {}
    # for resource in ranked_candidate_resources:
    #     print("resource", resource, file=sys.stderr)
    #     title, sentences = get_snippet(query, resource[0])
    #     results[resource[0]] = [title, sentences]
    ranked_candidate_queries = find_rank_candidate_queries(query)
    results["query_suggestions"] = []
    for query_suggestion in ranked_candidate_queries[:5]:
        results["query_suggestions"].append(query_suggestion)

    return results

search_results = search("woman")
print("results:")
print(search_results)

Identifying Candidate Queries...
aol query                                    amazon
                    academy award nominee
                     family tree template
                free family tree template
                        free family chart
               free blank geneology chart
                                     file
                           metro district
                     denver county police
                           metro district
                    nike woman sportswear
                                     nike
                                    woman
                    woman track suit nike
                                    woman
                      woman athletic wear
                               gart sport
                     nike fitness apparel
                  steamboat spring resort
                 parking steamboat spring
                                   goggle
                         galapagos iguana
                         galapago

TypeError: unhashable type: 'DataFrame'

In [1]:
# aol_query_log = pickle.load(open("./data/aol_query_log_data.pkl", "rb"))

aol_query_log.iloc[1065383]

NameError: name 'aol_query_log' is not defined